In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import requests
import os
from requests.exceptions import ConnectTimeout
from bs4 import BeautifulSoup

In [2]:
#una pajina dinamica, entonces Selenium
driver = webdriver.Chrome()
driver.get("https://www.argentina.gob.ar/anmat/normativa-general")

pdf_links = []

for i in range(36):
    time.sleep(5)
    
    #guardar botones
    buttons = driver.find_elements(By.CLASS_NAME, 'btn.btn-primary.btn-sm.margin-btn')
    pdf_links.extend([button.get_attribute('href') for button in buttons])
    
    #la pajina siguente
    next_button = driver.find_element(By.XPATH, '//a[@aria-label="Ir a la página siguiente"]')
    
    #no hay mas pajinas
    #if 'disabled' in next_button.get_attribute('class'):
    #    break
    
    next_button.click()

driver.quit()

In [3]:
os.makedirs('anmat', exist_ok=True)

In [18]:
download_dir = os.path.join(os.getcwd(), 'anmat')
chrome_options = Options()
chrome_prefs = {
    "profile.default_content_settings.popups": 0,
    "download.default_directory": download_dir,
    "directory_upgrade": True,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", chrome_prefs)

In [5]:
#descargar pdf
for pdf_link in pdf_links:
    if '.pdf' in pdf_link:
        pdf_response = requests.get(pdf_link)
        file_name = os.path.join('anmat', pdf_link.split("/")[-1])
        with open(file_name, 'wb') as file:
            file.write(pdf_response.content)
        print(f"{file_name}")

anmat\circular_02-16_productos_medicos_incumbencias_de_directores_tecnicos.pdf
anmat\circular_ms_8_16.pdf
anmat\anmat_circular_7_2013.pdf
anmat\anmat_circular_3_11.pdf
anmat\anmat_circular_1_11.pdf
anmat\Resolucion_462-10.pdf
anmat\Resolucion_405-2010.pdf
anmat\Resolucion_406-2010.pdf
anmat\anmat_circular_4_10.pdf
anmat\disposicion_anmat_1743_2009.pdf
anmat\anmat_disposicion_615_2007.pdf
anmat\disposicion_anmat_7398_2007.pdf
anmat\disposicion_anmat_3110_2007.pdf
anmat\disposicion_anmat_1955_2007.pdf
anmat\anmat_disposicion_1336_2006.pdf
anmat\Disposicion_2005_2060.pdf
anmat\anmat_disposicion_998_2004.pdf
anmat\anmat_disposicion_anmat_1507_2002.pdf
anmat\anmat_disposicion_2438_2000.pdf
anmat\anmat_disposicion_935_2000.pdf
anmat\disposicion_anmat_4306_1999.pdf
anmat\disposicion_anmat_2675_1999.pdf
anmat\disposicion_anmat_3409_1999.pdf
anmat\disposicion_anmat_1149_1997.pdf
anmat\Disposicion_ANMAT_1930-1995.pdf
anmat\disposicion_anmat_2305-1995.pdf
anmat\disposicion_anmat_1930_1995.pdf
anm

In [6]:
def download_documents(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', href=True)
    for link in links:
        href = link['href'].replace('blank:#', '').strip()
        if href.startswith('https://www.argentina.gob.ar/sites/default/files/'):
            file_name = os.path.join(download_dir, href.split('/')[-1])
            try:
                print(f"cargando: {href}")
                file_response = requests.get(href)
                file_response.raise_for_status()
                with open(file_name, 'wb') as file:
                    file.write(file_response.content)
                print(f" terminado: {file_name}")
            except Exception as e:
                print(f"error {href}: {e}")
        if href.startswith('https://www.argentina.gob.ar/normativa/nacional/'):
            response = requests.get(non_pdf_link+'/texto')
            soup = BeautifulSoup(response.text, 'html.parser')
            doc_div = soup.find('body')
            if doc_div:
                file_name = os.path.join('anmat', non_pdf_link.split("/")[-1] + '.txt')
                with open(file_name, 'w', encoding='utf-8') as file:
                    for string in doc_div.stripped_strings:
                        file.write(string + "\n")
            print(f"{file_name}")

In [10]:
for non_pdf_link in pdf_links:
    if '.pdf' not in non_pdf_link and 'boletinoficial' not in non_pdf_link:
        download_documents(non_pdf_link)

cargando: https://www.argentina.gob.ar/sites/default/files/dec_2126.pdf
 terminado: C:\Users\blank\Projects\RMT\nlpTP\anmat\dec_2126.pdf
cargando: https://www.argentina.gob.ar/sites/default/files/capitulo_i_caa.pdf
 terminado: C:\Users\blank\Projects\RMT\nlpTP\anmat\capitulo_i_caa.pdf
cargando: https://www.argentina.gob.ar/sites/default/files/anmat_capitulo_ii_establecactualiz_2023-09.pdf
 terminado: C:\Users\blank\Projects\RMT\nlpTP\anmat\anmat_capitulo_ii_establecactualiz_2023-09.pdf
cargando: https://www.argentina.gob.ar/sites/default/files/capitulo_iii_prod_alimenticiosactualiz_2023-05_1.pdf
 terminado: C:\Users\blank\Projects\RMT\nlpTP\anmat\capitulo_iii_prod_alimenticiosactualiz_2023-05_1.pdf
cargando: https://www.argentina.gob.ar/sites/default/files/capitulo_iv_envasesactualiz_2024-08.pdf
 terminado: C:\Users\blank\Projects\RMT\nlpTP\anmat\capitulo_iv_envasesactualiz_2024-08.pdf
cargando: https://www.argentina.gob.ar/sites/default/files/anmat_caa_capitulo_v_rotulacion_actualiz_2

In [19]:
non_pdf_link = 'https://www.boletinoficial.gob.ar/detalleAviso/primera/313423/20240906'
driver = webdriver.Chrome(options=chrome_options)                
driver.get(non_pdf_link) 

pdf_button = driver.find_element(By.CSS_SELECTOR, "div.panel-body[onclick^='descargarPDFAnexo']")
parent_element = pdf_button.find_element(By.XPATH, "..")
ActionChains(driver).move_to_element(parent_element).click().perform()
#print("se descarga el pdf")
time.sleep(5)

driver.quit()

In [ ]:
for non_pdf_link in pdf_links:
    if '.pdf' not in non_pdf_link and 'boletinoficial' in non_pdf_link:
        try:
            response = requests.get(non_pdf_link)
            soup = BeautifulSoup(response.text, 'html.parser')
            aviso_div = soup.find('div', {'class': 'row', 'id': 'detalleAviso'})
            if aviso_div:
                file_name = os.path.join(download_dir, non_pdf_link.split("/")[-1] + '.txt')
                with open(file_name, 'w', encoding='utf-8') as file:
                    for string in aviso_div.stripped_strings:
                        file.write(string + "\n")
                    print(f"{file_name}")

            #selenium para descargar pdf añadido si existe
            try:
                print('loading pdf:driver')
                driver = webdriver.Chrome(options=chrome_options)                 
                driver.get(non_pdf_link)            
                pdf_button = driver.find_element(By.CSS_SELECTOR, "div.panel-body[onclick^='descargarPDFAnexo']")
                parent_element = pdf_button.find_element(By.XPATH, "..")
                print('loading pdf:button')
                ActionChains(driver).move_to_element(parent_element).click().perform()
                time.sleep(5)
                driver.quit()
            except Exception:
                pass
        except ConnectTimeout:
            print(non_pdf_link, 'connection timeout')
            pass